In [74]:
# Tutorial: https://www.youtube.com/watch?v=dRcvJRmqFHQ&lc=UgyUDvoIA0A37EWWoC54AaABAg.9Aem3mXGQzz9Aep5dNrAY6
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd
import os

class ZillowScraper():
    results = []
    headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'zguid=23|%24d7ce356c-e3c5-4a48-85b3-61d0af33c3a1; zgsession=1|76ac70ba-62bc-4f77-81a1-b46b55a3f268; _ga=GA1.2.656681018.1593695925; _gid=GA1.2.1901059192.1593695925; zjs_user_id=null; zjs_anonymous_id=%22d7ce356c-e3c5-4a48-85b3-61d0af33c3a1%22; _pxvid=930c8230-bc66-11ea-9dd2-0242ac120007; _gcl_au=1.1.2002891296.1593696177; KruxPixel=true; DoubleClickSession=true; _pin_unauth=dWlkPU9EbG1aR0ptTTJVdFkySXpPUzAwWVdZMUxXSTFNVE10TnpFNE1EZGpOamxsWWpZNQ; _fbp=fb.1.1593696180071.236888825; KruxAddition=true; g_state={"i_p":1593716288950,"i_l":1}; JSESSIONID=0C394BB3DC7B91E479B72A328353817A; GASession=true; _pxff_bsco=1; AWSALB=ICqV1IKa+cj8ecut5WcXnUJCq3pP//RbqBzZ49F9UxNm0bLvluUKA3yHj1v2AbnyZ74OnEhFTZXZYd39kdbddeIij3ZzH2d20INkvIhGwFyGR+evOyhe+uUmhlil; AWSALBCORS=ICqV1IKa+cj8ecut5WcXnUJCq3pP//RbqBzZ49F9UxNm0bLvluUKA3yHj1v2AbnyZ74OnEhFTZXZYd39kdbddeIij3ZzH2d20INkvIhGwFyGR+evOyhe+uUmhlil; search=6|1596363321838%7Crect%3D43.45764218649565%252C-109.08936284375%252C30.85123360186431%252C-129.52393315625%26rid%3D9%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26days%3D90%26type%3Dhouse%26lt%3Dfsba%252Cfsbo%252Cfore%252Cnew%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%099%09%09%09%09%09%09; _px3=d4336234c4010c751aebc89c46325b0fb974e47215ce19445dfb759a629e81dc:X7e2f9pB1glpxKBrZ31UbYr/SXJFF+UxNWu+hiMsI+DQ7Mqle+9ez9SpkHaEa0n8KjNaAtb3oFQCfBoM9SkdFQ==:1000:AfuabQEQyo2l//8QKend7zOjb5sDW5Is+c6zQcm/upUYyYJc1jNQiyYnwroZOcBIjJBNJ8+vUKEJ3TlBoNaX/O7UbA0YkImLQb+VkAlAZWRj+w8lmqxZjshBeq6rrIg7GVsOSD+a1N8QtabSrjTIOF1SbQdGaZFzwVUu+aT6YiQ=; _uetsid=b093ce5e-93c4-0334-9c39-42ee7af2a929; _uetvid=e59e74ae-427f-984f-1765-cdc49ba1fa47',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    }
        
    def fetch(self, url, params):
        response = requests.get(url, headers=self.headers, params=params)
        print(response.status_code)
        return response
    
    def parse(self, response):
        content = BeautifulSoup(response, 'lxml')
        deck = content.find('ul', {'class': 'photo-cards photo-cards_wow photo-cards_short'})
        for card in deck.contents:
            script = card.find('script', {'type': 'application/ld+json'})
            if script:
                script_json = json.loads(script.contents[0])
                loop_response = requests.get(script_json['url'], headers=self.headers)
                loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
                try:
                    loop_result2 = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[1].findAll('li', class_="ds-home-fact-list-item")
                except IndexError:
                    loop_result2 = 'null'
                lot = 'null';
                house_type = 'null';
                year_build = 'null';
                parking = 'null';
                hoa = 'null';
                lot_conversion_to_sqft = 'null';
                zillow_price_per_sqft = 'null';
                
                if loop_result2:
                    for contents in loop_result2:
                        print(contents.text)
                        if "Lot" in contents.text.split(":")[0]:
                            lot = contents.text.split(":")[1];
                        if "Type" in contents.text.split(":")[0]:
                            house_type = contents.text.split(":")[1];
                        if "Year built" in contents.text.split(":")[0]:
                            year_build = contents.text.split(":")[1];
                        if "Parking" in contents.text.split(":")[0]:
                            parking = contents.text.split(":")[1];
                        if "HOA" in contents.text.split(":")[0]:
                            hoa = contents.text.split(":")[1];
                        if "Price/sqft" in contents.text.split(":")[0]:
                            zillow_price_per_sqft = contents.text.split(":")[1];
                
                    try:
                        zestimate = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[2].find('p', class_="Text-aiai24-0 sc-cMljjf sc-cTjmhe eXGbuO").text
                        rent_zestimate = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[3].find('p', class_="Text-aiai24-0 sc-cMljjf sc-cTjmhe eXGbuO").text
                    except AttributeError:
                        zestimate = 'null'
                        rent_zestimate = 'null'
                    try:
                        if (str(lot.split(" ")[1]) == "sqft"):
                            lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                        elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                            lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                        elif (lot == "No Data"):
                            lot_conversion_to_sqft = 'null'
                        else:
                            lot_conversion_to_sqft = 'null'
                    except IndexError:
                        lot_conversion_to_sqft = 'null'
                        
                    self.results.append({
                        'Listing_type': card.find('div', {'class': 'list-card-type'}).text,
                        'Price': card.find('div', {'class': 'list-card-price'}).text,
                        'Full_address': script_json['name'],
                        'Street': script_json['address']['streetAddress'],
                        'City': script_json['address']['addressLocality'],
                        'State': script_json['address']['addressRegion'],
                        'Zip_code': script_json['address']['postalCode'],
                        'Latitude': script_json['geo']['latitude'],
                        'Longitude': script_json['geo']['longitude'],
                        'Square_Feet': script_json['floorSize']['value'],
                        'Lot': lot,
                        'Lot_conversion_to_sqft': lot_conversion_to_sqft,
                        'House_type': house_type,
                        'Year_build': year_build,
                        'Parking': parking,
                        'Hoa': hoa,
                        'Zillow_price_per_sqft': zillow_price_per_sqft,
                        'Zestimate': zestimate,
                        'Rent_zestimate': rent_zestimate,
                        'House_website': script_json['url']
                    })
                    time.sleep(3)
                
    def to_csv(self):
        dirName = 'Output/Zillow/'
        if not os.path.exists(dirName):
            os.mkdir(dirName)
            print("Directory ", dirName, " Folder created ")
        else:
            print("Directory ", dirName, " Folder already exists")
            
        with open('./' + dirName + 'Q2-2020_Single_Family_Data_page17-20.csv', 'w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())
            writer.writeheader()
            
            for row in self.results:
                writer.writerow(row)

    def to_dftable(self):
        house_data = pd.DataFrame(self.results)
        print(house_data)

#     Filter Out Coming Soon, Auction, Home Type: Houses, Days On Zillow: 90 Days
    def run(self):
        url = 'https://www.zillow.com/ca/houses/'
        for page in range(17,21):
            params = {
                'searchQueryState':'{"pagination":{"currentPage":%s},"usersSearchTerm":"ca","mapBounds":{"west":-130.9521558125,"east":-107.6611401875,"south":28.570940002914515,"north":45.333552921012235},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":5,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"}},"isListVisible":true}' %page
            }
            res = self.fetch(url, params)
            self.parse(res.text)
            time.sleep(2)
        self.to_dftable()
        self.to_csv()
        
if __name__ == '__main__':
    scraper = ZillowScraper()
    scraper.run();

200
Type:Single Family
Year built:1978
Heating:Forced air
Cooling:Central
Parking:1 space
HOA:$325/month
Lot:2.63 acres
Price/sqft:$13,206
Type:Single Family
Year built:1963
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:7,562 sqft
Price/sqft:$338
Type:Single Family
Year built:2005
Heating:Forced air
Cooling:Central
Parking:3 spaces
HOA:$208/month
Lot:9,147 sqft
Price/sqft:$223
Type:Single Family
Year built:1955
Heating:Forced air
Cooling:Central
Parking:1 space
HOA:None
Lot:6,098 sqft
Price/sqft:$237
Type:Single Family
Year built:1957
Heating:Forced air
Cooling:Central
Parking:No Data
Lot:4,412 sqft
Price/sqft:$956
Type:Single Family
Year built:2018
Heating:No Data
Cooling:Other
Parking:2 spaces
HOA:None
Lot:2,556 sqft
Price/sqft:$695
Type:Single Family
Year built:1944
Heating:Forced air
Cooling:Central
Parking:5 spaces
HOA:Yes
Lot:6,046 sqft
Price/sqft:$732
Type:Single Family
Year built:1986
Heating:Wall
Cooling:No Data
Parking:2 spaces
Lot:0.55 acres
Price/sqft:$20

Type:Single Family
Year built:1981
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:0.35 acres
Price/sqft:$295
Type:Single Family
Year built:1994
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:4,499 sqft
Price/sqft:$232
Type:Single Family
Year built:2006
Heating:Forced air
Cooling:Central
Parking:4 spaces
HOA:$100/month
Lot:6,534 sqft
Price/sqft:$273
Type:Single Family
Year built:1964
Heating:Forced air
Cooling:None
Parking:2 spaces
HOA:$264/month
Lot:1,585 sqft
Price/sqft:$494
Type:Single Family
Year built:1942
Heating:Forced air
Cooling:None
Parking:3 spaces
Lot:7,191 sqft
Price/sqft:$449
Type:Single Family
Year built:2016
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:$159/month
Lot:0.67 acres
Price/sqft:$271
Type:Single Family
Year built:2017
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:5,475 sqft
Price/sqft:$548
Type:Single Family
Year built:1988
Heating:Forced air
Cooling:Central
Parking:6 spaces
HOA:$108/month
Lot:5,

Type:Single Family
Year built:1960
Heating:Forced air
Cooling:Evaporative
Parking:1 space
HOA:None
Lot:0.29 acres
Price/sqft:$128
Type:Single Family
Year built:1974
Heating:Other
Cooling:Central
Parking:2 spaces
Lot:4,948 sqft
Price/sqft:$111
Type:Single Family
Year built:1978
Heating:Forced air
Cooling:Central
Parking:10 spaces
Lot:10,018 sqft
Price/sqft:$272
Type:Single Family
Year built:2008
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:$150/month
Lot:0.31 acres
Price/sqft:$356
Type:Single Family
Year built:1976
Heating:Other
Cooling:Central
Parking:Attached Garage
HOA:None
Lot:6,534 sqft
Price/sqft:$292
Type:Single Family
Year built:1987
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:4,552 sqft
Price/sqft:$191
Type:Single Family
Year built:2001
Heating:Forced air
Cooling:Central
Parking:2 spaces
HOA:None
Lot:5,662 sqft
Price/sqft:$280
Type:Single Family
Year built:1950
Heating:Forced air
Cooling:Central
Parking:Carport
HOA:None
Lot:8,276 sqft
Price/sqft: